In [ ]:
import sys
sys.path.append('../lsnc/')
from hierarchical_lsnc import HierarchicalLSNC
import numpy as np
from tqdm import tqdm
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import itertools


## import the list of datasets
datasets = []
for directory in os.listdir("./labeled-datasets/npy/"):
	if (
		directory != ".DS_Store"
		and directory != "README.md"
		and directory != ".gitignore"
		and not directory.endswith(".zip")
	):
		datasets.append(directory)

## load the data
raw_list     = []
label_list   = []
tsne_list    = []
umap_list    = []
pca_list	   = []
iso_list	   = []
lle_list     = []
densmap_list = []

for dataset in tqdm(datasets):
	raw = np.nan_to_num(np.load(f"./labeled-datasets/npy/{dataset}/data.npy"))
	raw = (raw - raw.mean(axis=0)) / raw.std(axis=0)
	raw = raw[:, ~np.isnan(raw).any(axis=0)]
	raw_list.append(raw)
	label_list.append(np.load("./labeled-datasets/npy/{}/label.npy".format(dataset)))
	tsne_list.append(np.load("./labeled-datasets_embedding/{}/tsne.npy".format(dataset)))
	umap_list.append(np.load("./labeled-datasets_embedding/{}/umap.npy".format(dataset)))
	pca_list.append(np.load("./labeled-datasets_embedding/{}/pca.npy".format(dataset)))
	iso_list.append(np.load("./labeled-datasets_embedding/{}/isomap.npy".format(dataset)))
	lle_list.append(np.load("./labeled-datasets_embedding/{}/lle.npy".format(dataset)))
	densmap_list.append(np.load("./labeled-datasets_embedding/{}/densmap.npy".format(dataset)))




In [ ]:
def runEmbeddings(emb_list):
    frames = []
    for idx, (raw, emb, label) in enumerate(zip(raw_list, emb_list, label_list)):
        print(f'Dataset: {idx}')
        #if idx != 0: continue
        try: 
            hlsnc = HierarchicalLSNC(raw, emb)
            result = hlsnc.run(granularity=10)
            frames.append({
                "dataset": idx,
                "ls": result["ls"],
                "lc": result["lc"],
                "unique_raw_label": np.unique(label).tolist()
            })
        except: 
            print(f"Error in: {idx}")
    return frames

In [ ]:
dr_types = ['tsne', 'umap', 'pca', 'iso', 'lle', 'densmap']
embs = [tsne_list, umap_list, pca_list, iso_list, lle_list, densmap_list]

frames = []
for dr_type, emb in zip(dr_types, embs):
    results = runEmbeddings(emb)
    frames.append({
        'result': results,
        'dr_type': dr_type,
    })

In [ ]:
import json
with open('./shc_data/data_granularity_10.json', 'w') as f:
    json.dump(frames, f)

In [ ]:
'''
Compare t-SNE / UMAP / PCA / Isomap / LLE / Densmap from using hierarchical LSNC (already imported)
aggregate the results as a single line for each technique 
(use lineplot in seaborn)

the final result should be a lineplot with 6 lines and confidence intervals (for each technique)
'''
def _reformat(nestedData: pd.DataFrame, column_name: str):
    decouples = nestedData[column_name].apply(pd.Series)
    decouples = pd.melt(decouples, value_vars=decouples.columns, var_name='level', value_name='score')
    decouples = decouples.loc[~decouples['score'].isna(), :]
    decouples['metric'] = column_name
    return decouples

stats = []
for each in frames:
    data = pd.DataFrame(each.get('result', None))
    data_ls = _reformat(data, 'ls')
    data_lc = _reformat(data, 'lc')
    data = pd.concat([data_ls, data_lc], ignore_index=True)
    data['type'] = each.get('dr_type', '')
    stats.append(data)

stats = pd.concat(stats, ignore_index=True)

In [ ]:
plt.clf()
sns.set_style("whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
dr_types = ['tsne', 'umap', 'pca', 'iso', 'lle', 'densmap']
dr_names = ['t-SNE', 'UMAP', 'PCA', 'ISO', "LLE", 'Densmap']
metrics = ['ls', 'lc']
colors = [sns.color_palette('tab10')[i] for i in range(6)]

for i, metric in enumerate(metrics):
	mask = (stats['metric'] == metric)
	sns.lineplot(x = "level", y = "score", hue = "type", data = stats.loc[mask, :], ax = axes[i%2])

	## set x axis log
	#axes[i%2].set_xscale("log")

	## set label with measures_name
	axes[i%2].set_xlabel("Hierarchical Clustering Granularity")
	axes[i%2].set_ylabel("Score" if i == 0 else "")
	axes[i%2].set_title("Label-Stretching" if i == 0 else "Label-Compression")
	## set legend
	if i== 0:
		axes[i].legend(labels=dr_names, title=None)
	else:
		axes[i].legend([],[], frameon=False)
